In [ ]:
import requests
from collections import defaultdict
import re

In [ ]:
response = requests.get("https://www.khanacademy.org/api/internal/translate/progress_nodes?lang=de&use_stage_for_origin_nodes=0&_=1516120716805")

In [ ]:
obj = response.json()

In [ ]:
obj

In [ ]:
list(obj["nodes"].keys())

In [ ]:
exids = [topic["slug"] for topic in obj["nodes"]["exercises"].values() if topic["contentKind"] == "Exercise"]
exids

In [ ]:

def raw_assessment_info(exid):
    response = requests.get("https://www.khanacademy.org/api/internal/exercises/{}/assessment_items?lang=de&localized_topic_tree=&_=1515982197307".format(exid))
    if response.text == "API error":
        return None
    return response.json()

def assignment_info(exid):
    ai = raw_assessment_info(exid)
    # Build assignment ID ("x844a80c198728eed") to problem type ("Type 1") map
    problemTypeMap = {}
    for pt, assIDs in ai["problemTypes"].items():
        for assID in assIDs:
            problemTypeMap[assID] = pt
    # Build translation items by assignment
    assignmentStrings = defaultdict(set)
    for ti in ai["translationItems"]:
        # Usually (or always?) each ID occurs twice
        crowdinIDs = set(re.findall(r"crwdn[se](\d+)", ti["jiptString"]))
        assignmentStrings[ti["assessmentItem"]].update(crowdinIDs)
    return problemTypeMap, assignmentStrings

assignment_info("skip-counting-by-10s")